In [3]:
!apt install -y python-opengl ffmpeg > /dev/null 2>&1
%pip install pyvirtualdisplay 

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install \
gym==0.25.2 \
gymnasium==1.0.0 \
imutils==0.5.4 \
Jinja2==3.1.4 \
joblib \
libclang==18.1.1 \
Markdown==3.7 \
MarkupSafe==3.0.2 \
matplotlib==3.9.3 \
panda-gym==3.0.7 \
pillow==11.0.0 \
pybullet==3.2.6 \
six==1.16.0 \
sympy==1.13.1 \



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 11.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 13.6 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 79.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852297 sha256=9d7eb1b354607b6b62db474d224db226b0354b13d6c04822315519ed81fe3e5a
  Stored in directory: /root/.cache/pip/wheels/78/95/2c/ee47a8d43fda6a851e340

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()


from matplotlib import pyplot as plt, animation
%matplotlib inline
from IPython import display

def create_anim(frames, dpi, fps):
    plt.figure(figsize=(frames[0].shape[1] / dpi, frames[0].shape[0] / dpi), dpi=dpi)
    patch = plt.imshow(frames[0])
    def setup():
        plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, init_func=setup, frames=len(frames), interval=fps)
    return anim

def display_anim(frames, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    return anim.to_jshtml()

def save_anim(frames, filename, dpi=72, fps=50):
    anim = create_anim(frames, dpi, fps)
    anim.save(filename)


class trigger:
    def __init__(self):
        self._trigger = True

    def __call__(self, e):
        return self._trigger

    def set(self, t):
        self._trigger = t

In [6]:
import pybullet as p
# Add a marker for the world frame
def add_world_frame():
    length = 0.1  # Length of the axis
    radius = 0.01  # Radius of the lines

    print("Frame")
    print("X-axis: red")
    print("Y-axis: green")
    print("Z-axis: blue")

    # X-axis (red)
    p.addUserDebugLine([0, 0, 0], [length, 0, 0], [1, 0, 0], lineWidth=2)

    # Y-axis (green)
    p.addUserDebugLine([0, 0, 0], [0, length, 0], [0, 1, 0], lineWidth=2)

    # Z-axis (blue)
    p.addUserDebugLine([0, 0, 0], [0, 0, length], [0, 0, 1], lineWidth=2)


pybullet build time: Nov 28 2023 23:45:17


In [7]:
import math
from math import pi
from random import choice, uniform
from typing import Any, Dict, Tuple, Optional

import numpy as np
from panda_gym.envs.core import RobotTaskEnv, Task
from panda_gym.envs.robots.panda import Panda
from panda_gym.pybullet import PyBullet


class TargetObject:
    """
    TargetObject tracks the lifecycle of a target object (not a goal)
    """

    def __init__(
        self,
        id: str,
        name: str,
        shape: int,
        size: np.array,
        position: np.array,
        color: np.array,
        removed: bool = False,
    ):
        self.id = id
        self.name = name
        self.shape = shape
        self.size = size
        self.position = position
        self.color = color
        self.removed = removed


class Pick_And_Place(Task):
    def __init__(
        self,
        sim: PyBullet,
        observation_type: int,
        robot: Panda,
        objects_count: int = 3,
        img_size: Tuple[int, int] = (256, 256),
        blocker_bar: bool = True,
        sorting_count: int  = 1
    ):
        if observation_type not in [OBSERVATION_POSES, OBSERVATION_IMAGE]:
            raise Exception(
                f"Invalid output type {observation_type}. Must be one of "
                + f"{OBSERVATION_POSES} for values and {OBSERVATION_IMAGE} "
                + "four images."
            )

        super().__init__(sim)

        self.robot = robot
        self.observation_type = observation_type
        self.sorting_count = sorting_count
        self.score: float = 0.0

        self.objects_count: int = objects_count
        if observation_type == OBSERVATION_IMAGE:
            self.img_size = img_size
        self.object_opacity = 0.8

        self.sim.create_plane(z_offset=-0.4)

        # goal_positions will contain one of the three
        # preset strings (set below) w/ the resulting
        # starting position each goal is expected to be
        self.sorter_positions: Dict[str, np.array] = {}
        self.blocker_bar = blocker_bar
        self._init_sorting_areas()

        # Track each target object as part of our goal
        self.goal: Dict[int, TargetObject] = {}

        # Size multiplier is the range of sizes allowed for
        # target objects
        self.size_multiplier: Tuple[float, float] = (0.5, 1)

        self.task_init()

    def task_init(self):
        # Create our plane and table for the scenario
        self.sim.create_table(length=0.8, width=0.8, height=0.4, x_offset=-0.3)

        # These position_limits are where the objects are allowed
        # to spawn. This reads as (x, y), where each axis
        # in turn is a tuple of min/max placement.
        self.object_position_limits: Tuple[Tuple[float, float]] = (
            (-0.06, 0.06),
            (-0.2, 0.2),
        )
        # self.sim.create_sphere(
        #     body_name="marker",
        #     radius=0.025,
        #     mass=0.0,
        #     ghost=True,
        #     position=(0.06, 0.2, 0.01),
        #     rgba_color=(255, 255, 0, 1.0),
        # )

    def _init_sorting_areas(self):
        if self.sorting_count == 3:
            self.sorter_positions = {
                SORTING_ONE: np.array([-0.25, -0.2, 0.01]),
                SORTING_TWO: np.array([-0.25, 0.00, 0.01]),
                SORTING_THREE: np.array([-0.25, 0.2, 0.01]),
            }
        if self.sorting_count == 2:
            self.sorter_positions = {
                SORTING_ONE: np.array([-0.25, -0.2, 0.01]),
                SORTING_TWO: np.array([-0.25, 0.00, 0.01]),
            }
        if self.sorting_count == 1:
            self.sorter_positions = {
                SORTING_ONE: np.array([-0.25, -0.2, 0.01]),
            }
        if self.blocker_bar:
            self.sorter_positions["blocker"] = np.array([-0.2, 0.0, 0.01])
        count = self.sorting_count
        if (count == 3):
            self.sim.create_box(
                body_name=SORTING_THREE,
                half_extents=np.array([0.05, 0.1, 0.01]),
                mass=0.0,
                ghost=False,
                position=self.sorter_positions[SORTING_THREE],
                rgba_color=np.array([1.0, 0, 0, 0.4]),
            )
            count -=1
        if count == 2:
            self.sim.create_box(
                body_name=SORTING_TWO,
                half_extents=np.array([0.05, 0.1, 0.01]),
                mass=0.0,
                ghost=False,
                position=self.sorter_positions[SORTING_TWO],
                rgba_color=np.array([0.0, 1.0, 0, 0.4]),
            )
            count-=1
        if count == 1:
            self.sim.create_box(
                body_name=SORTING_ONE,
                half_extents=np.array([0.05, 0.1, 0.01]),
                mass=0.0,
                ghost=False,
                position=self.sorter_positions[SORTING_ONE],
                rgba_color=np.array([0, 0, 1.0, 0.5]),
            )

        if self.blocker_bar:
            # Create the blocking bar
            self.sim.create_box(
                body_name="blocker",
                half_extents=np.array([0.01, 0.3, 0.005]),
                mass=0.0,
                ghost=False,
                position=self.sorter_positions["blocker"],
                rgba_color=np.array([0.0, 0.0, 0.0, 0.8]),
            )

    def set_sorter_positions(self):
        """
        set_goal_positions will ensure that goals are placed
        in the appropriate place in the environment
        """
        # for count in range(self.sorting_count):
        #     self.sim.set_base_pose(
        #         GOALS[count],
        #         position=self.sorter_positions[GOALS[count]],
        #         orientation=np.array([0.0, 0.0, 0.0, 1.0]),
        #     )
        for sorter in self.sorter_positions:
            self.sim.set_base_pose(
                sorter,
                position=self.sorter_positions[sorter],
                orientation=np.array([0.0, 0.0, 0.0, 1.0]),
            )

    def setup_target_objects(self):
        """
        Generate self.objects_count objects randomly on the table of
        varying sizes, colors, and shapes. The shapes check to ensure
        that they do NOT collide with one another to start.
        """
        base_size = 0.025
        base_mass = 0.5
        base_box_volume = base_size**3
        base_cylinder_volume = pi * base_size**3  # h == r in base cylinder
        # First, delete each object to cleanup
        self.delete_all_objects()

        for object in range(0, self.objects_count):
            # Attempt to create the object. If it collides with
            # another, delete it and try again
            while True:
                name = f"object_{object}"
                color = self.get_random_color()
                if (self.sorting_count == 1):
                    shape = choice(SHAPES[0:1])
                if (self.sorting_count == 2):
                    shape = choice(SHAPES[0:2])
                if (self.sorting_count == 3):
                    shape = choice(SHAPES)
                position = self.get_random_object_position()

                if shape == CUBE:
                    x = base_size * uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    y = base_size * uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    z = base_size * uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    size = np.array([x, y, z]).astype(np.float32)

                    volume = x * y * z
                    mass_multiplier = volume / base_box_volume

                    self.sim.create_box(
                        body_name=name,
                        half_extents=np.array([x, y, z]),
                        mass=base_mass * mass_multiplier,
                        ghost=False,
                        position=position,
                        rgba_color=color,
                    )
                elif shape == CYLINDER:
                    height = base_size * uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    radius = base_size * uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    size = np.array([height, radius, 0.0]).astype(np.float32)

                    volume = pi * radius**2 * height
                    mass_multiplier = volume / base_cylinder_volume

                    self.sim.create_cylinder(
                        body_name=name,
                        radius=radius,
                        height=height,
                        mass=base_mass * mass_multiplier,
                        ghost=False,
                        position=position,
                        rgba_color=color,
                    )
                elif shape == SPHERE:
                    multiplier = uniform(
                        self.size_multiplier[0], self.size_multiplier[1]
                    )
                    size = np.array([multiplier, 0.0, 0.0]).astype(np.float32)

                    self.sim.create_sphere(
                        body_name=name,
                        radius=base_size * multiplier,
                        mass=base_mass * multiplier,
                        ghost=False,
                        position=position,
                        rgba_color=color,
                    )
                else:
                    raise Exception("Improper shape chosen")

                id = self.sim._bodies_idx[name]

                # Now ensure that the shape created does not
                # intersect any of the existing shapes
                collisions = False
                # If this is the first, we're good; move on
                if len(self.goal) <= 0:
                    break
                # ...otherwise we're going to compare it
                # against all known objects. If there's
                # overlap we delete this and move on
                for other in self.goal:
                    other_id = self.goal[other].id
                    if self.check_collision(id, other_id):
                        collisions = True
                        break

                if collisions:
                    self.sim.physics_client.removeBody(id)
                    continue
                else:
                    break

            self.goal[object] = TargetObject(id, name, shape, size, position, color)

    def check_collision(self, object1: str, object2: str) -> bool:
        """
        check_collision will check if the two objects overlap at all
        and returns a boolean to that effect
        """
        contacts = self.sim.physics_client.getContactPoints(object1, object2)
        return contacts is not None and len(contacts) > 0

    def delete_all_objects(self):
        for object in self.goal:
            self.sim.physics_client.removeBody(self.goal[object].id)
        self.goal = {}

    def get_random_color(self) -> np.array:
        """
        Returns an appropriate color from a list of decent color choices
        in the form of a 4 dimensional RGBA array (colors are (0,255) ->
        (0, 1) scaled)
        """
        colors = [
            (255, 0, 0),
            (0, 255, 0),
            (0, 0, 255),
            (255, 0, 255),
            (178, 102, 255),
            (102, 255, 255),
            (102, 0, 204),
            (255, 128, 0),
            (204, 0, 102),
        ]
        color = choice(colors)

        return np.array([color[0], color[1], color[2], self.object_opacity])

    def get_random_object_position(self) -> np.array:
        """
        get_random_object_position returns a random np.array of an object's
        permissions within the permissive bounds set at instantiation.
        """
        x = uniform(
            self.object_position_limits[0][0], self.object_position_limits[0][1]
        )
        y = uniform(
            self.object_position_limits[1][0], self.object_position_limits[1][1]
        )
        z = 0.01
        return np.array([x, y, z])

    def reset(self):
        # Ensure each goal hasn't moved
        self.set_sorter_positions()

        # Generate new objects
        self.setup_target_objects()

        # Clear our score
        self.score = 0.0

    def get_obs(self) -> Tuple[np.array, float]:
        """
        Determines if any objects collided, adjusts score and reward accordingly,
        and returns an observation along with the reward for this step.

        Returns:
            np.array: The observation at this step.
            float: The reward for this step.
        """
        reward = 0.0  # Initialize the reward

        # Handle floor collisions
        reward += self._handle_floor_collisions()

        # Handle goal collisions
        reward += self._handle_goal_collisions()

        # Reward for moving towards the closest object
        reward += self._reward_closer_to_object()

        # Reward for successful grasping
        #reward += self._reward_grasping_success()

        # Reward for moving an object towards its goal
        reward += self._reward_object_towards_goal()

        # Penalize time-step to encourage efficiency
        reward += STEP_PENALTY

        # Ensure that each goal stays in position
        self.set_sorter_positions()

        # Return the observation and the reward
        if self.observation_type == OBSERVATION_IMAGE:
            observation = self._get_img()
        else:
            observation = self._get_poses_output()
        self.score += reward
        return observation, reward

    def _handle_floor_collisions(self) -> float:
        """Checks for floor collisions and penalizes accordingly."""
        reward = 0.0
        floor_id = self.sim._bodies_idx["plane"]
        for object_key in self.goal:
            if self.goal[object_key].removed:
                continue

            object_id = self.goal[object_key].id
            if self.check_collision(object_id, floor_id):
                reward += FLOOR_COLLISION_PENALTY
                self.sim.physics_client.removeBody(object_id)
                self.goal[object_key].removed = True
                print(f"Object {object_key} dropped to the floor")
        return reward

    def _handle_goal_collisions(self) -> float:
        """Checks for collisions between objects and goals, rewarding or penalizing as necessary."""
        reward = 0.0
        for object_key in self.goal:
            if self.goal[object_key].removed:
                continue

            for i in range(self.sorting_count):
                goal = GOALS[i]
                object = self.goal[object_key]
                object_id = object.id
                goal_id = self.sim._bodies_idx[goal]

                if self.check_collision(object_id, goal_id):
                    self.sim.physics_client.removeBody(object_id)
                    self.goal[object_key].removed = True

                    # Reward or penalize based on correct/incorrect sorting
                    if CORRECT_SORTS[goal] == object.shape:
                        reward += DROP_SUCCESS_REWARD
                        print(f"Object {object_key} correctly sorted into {goal}")
                    else:
                        reward += WRONG_DROP_PENALTY
                        print(f"Object {object_key} incorrectly sorted into {goal}")
        return reward

    def _reward_closer_to_object(self) -> float:
        """Rewards the agent for moving closer to the nearest object."""
        ee_position = self.robot.get_ee_position()
        closest_object, closest_distance = self._get_closest_object(ee_position)
        if closest_object:
            distance_to_object = np.linalg.norm(ee_position - self.get_object_pose(closest_object)[:3])
            distance_delta = abs(closest_distance - distance_to_object)
            return MOVE_TOWARD_OBJECT_REWARD * distance_delta
        return 0.0

    def _reward_grasping_success(self) -> float:
        """Rewards the agent for successfully grasping an object."""
        return 0.0
        closest_object, _ = self._get_closest_object(self.robot.get_ee_position())
        if closest_object and self._is_object_grasped(closest_object):
            print(f"Object {closest_object} successfully grasped")
            return GRASP_SUCCESS_REWARD

    def _reward_object_towards_goal(self) -> float:
        """Rewards the agent for moving objects closer to their goals."""
        closest_object, closest_distance = self._get_closest_object(self.robot.get_ee_position())
        if closest_object and not closest_object.removed:
            object_pos = self.get_object_pose(closest_object)[:3]
            goal_pos = self.sorter_positions[GOALS[closest_object.shape]]
            distance_to_goal = np.linalg.norm(object_pos - goal_pos)
            distance_delta = abs(closest_distance - distance_to_goal)
            return MOVE_OBJECT_TO_GOAL_REWARD * distance_delta

        return 0.0

    def _get_closest_object(self, ee_position: np.array) -> Tuple[Optional[TargetObject], float]:
        """ Return the closest object to the end-effector (EE). """
        closest_object = None
        closest_distance = float('inf')
        for object in self.goal.values():
            if object.removed:
                continue
            object_pos = self.get_object_pose(object)[:3]  # x, y, z position
            distance = np.linalg.norm(ee_position - object_pos)
            if distance < closest_distance:
                closest_object = object
                closest_distance = distance
        return closest_object, closest_distance

    def _is_object_grasped(self, object) -> bool:
        """ Check if the gripper has grasped an object. """
        return self.robot.get_fingers_width() < GRASP_THRESHOLD

    def get_object_pose(self, object: TargetObject) -> np.array:
        object_position = self.sim.get_base_position(object.name)
        object_rotation = self.sim.get_base_rotation(object.name)
        object_velocity = self.sim.get_base_velocity(object.name)
        object_angular_velocity = self.sim.get_base_angular_velocity(object.name)
        observation = np.concatenate(
            [object_position, object_rotation, object_velocity, object_angular_velocity]
        )
        return observation.astype(np.float32)



    def _get_poses_output(self) -> np.array:
        """
        _get_poses_output will return the poses of all objects in the scene,
        as well as their identity and size. It will be a series of values for
        the raw (x, y, z, theta, phi, psi) pose of the object, as well as an
        identity (type of shape), and size (0-1) for min/max size, and the pose
        of the robot's end effector, and a 0-1 value for its gripper open/close
        state. An example of the return would be:

        [[x, y, z, theta, phi, psi,
          xd, yd, zd, thetad, phid, psid, <~ velocities
        [identity], [size]] (times # of set objects), ...,
        (ee_x, ee_y, ee_z, ee_theta, ee_phi, ee_psi,
        ee_xd, ee_yd, ee_zd, ee_thetad, ee_phid, ee_psid), # <~ velocities
        gripper_status]

        Note that the identity is a one-hot encoded list of shape [CUBE, CYLINDER,
        SPHERE] and that size is a three value array of varying meaning based
        on size: [x, y, z] for CUBE, [radius, height] for CYLINDER, [radius]
        for SPHERE. Unused values are 0.0.
        """
        # The size of this vector is determined by the number of objects expected
        pose_values = 12
        shape_values = 3
        size_values = 3
        # End effector values
        ee_values = 12
        finger_values = 1
        # The length of our vector is (pose_values + (identity, size)) for each
        # object, pose_values for the end effector, and one additional value for
        # the gripper finger state (distance between fingers)
        size = (
            (len(self.goal) * (pose_values + shape_values + size_values))
            + ee_values
            + finger_values
        )
        observation: np.array = np.zeros((size,), dtype="float32")

        index = 0
        for object in self.goal.values():
            # If the object has been removed, just report 0's for its existence
            if object.removed:
                index += 1
                continue

            pose = self.get_object_pose(object)
            object_index = index * (pose_values + shape_values + size_values)
            observation[object_index : object_index + pose_values] = pose

            # The shape is a one hot encoded vector of [CUBE, CYLINDER, SPHERE]
            if object.shape == CUBE:
                shape_type = [1, 0, 0]
            elif object.shape == CYLINDER:
                shape_type = [0, 1, 0]
            elif object.shape == SPHERE:
                shape_type = [0, 0, 1]
            shapes_index = object_index + pose_values
            observation[shapes_index : shapes_index + shape_values] = shape_type
            size_index = shapes_index + shape_values
            observation[size_index : size_index + size_values] = object.size
            index += 1

        # Get the end effector position
        ee_position = self.robot.get_ee_position()
        ee_rotation_quaternion = self.sim.get_link_orientation(
            self.robot.body_name, self.robot.ee_link
        )
        ee_rotation = self._quaternion_to_euler(ee_rotation_quaternion)
        # print("rot", ee_rotation)
        # print("rot other", self.sim.get_base_rotation(self.robot.ee_link))
        ee_velocity = self.robot.get_ee_velocity()
        ee_rotational_velocity = self.sim.get_link_angular_velocity(
            self.robot.body_name, self.robot.ee_link
        )

        # ee_angulary_velocity = 0.0
        fingers_width = self.robot.get_fingers_width()
        ee_index = (pose_values + shape_values + size_values) * len(self.goal)
        observation[ee_index : ee_index + 3] = ee_position
        observation[ee_index + 3 : ee_index + 6] = ee_rotation
        observation[ee_index + 6 : ee_index + 9] = ee_velocity
        observation[ee_index + 9 : ee_index + 12] = ee_rotational_velocity
        observation[ee_index + 12] = fingers_width

        return observation

    def _quaternion_to_euler(self, quaternion: np.array):
        """
        _quaternion_to_euler will convert a quaternion to euler angless
        """
        x, y, z, w = quaternion
        t0 = 2.0 * (w * x + y * z)
        t1 = 1.0 - 2.0 * (x * x + y * y)
        X = math.atan2(t0, t1)

        t2 = 2.0 * (w * y - z * x)
        t2 = 1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        Y = math.asin(t2)

        t3 = 2.0 * (w * z + x * y)
        t4 = 1.0 - 2.0 * (y * y + z * z)
        Z = math.atan2(t3, t4)

        return np.array([X, Y, Z]).astype(np.float32)

    def _get_img(self) -> np.array:
        """
        _get_img will return the image from the camera in human rendering
        mode
        """
        # We have to swap render mode if it's set to human mode
        # to get it to draw for us.
        original_render_mode = self.sim.render_mode
        self.sim.render_mode = "rgb_array"
        img = self.sim.render(
            self.img_size[0],
            self.img_size[1],
            # target_position=self.camera_position,
            target_position=None,
            distance=0.0,
            yaw=45,
            pitch=-30,
            roll=0.0,
        )
        self.sim.render_mode = original_render_mode
        return img

    def get_achieved_goal(self) -> np.ndarray:
        return np.array(
            all(target.removed for target in self.goal.values()), dtype="bool"
        )

    def is_terminated(self) -> bool:
        """
        is_terminated returns whether or not the episode is
        in a terminal state; this can be due to:
        1. All objects have been removed somehow from the env
        2. The timer has hit 0

        It is not an indication of success
        """

        return all(obj.removed for obj in self.goal.values())

    def is_success(
        self,
        achieved_goal: np.ndarray,
        desired_goal: np.ndarray,
        info: Dict[str, Any] = ...,
    ) -> np.ndarray:
        """
        is_success is a misnamed function, required as a layover
        from using the panda_gym library. Instead it is best
        to read it as an interface w/ is_terminated, and in no
        way reads whether it was a success, since the episode can
        end via timeout without doing the goals.
        """
        return np.array([self.is_terminated()], dtype="bool")

    def compute_reward(
        self,
        achieved_goal: np.ndarray,
        desired_goal: np.ndarray,
        info: Dict[str, Any] = ...,
    ) -> np.ndarray:
        return np.array([self.score], dtype="float32")


class My_Arm_RobotEnv(RobotTaskEnv):
    """Sorter task wih Panda robot.

    Args:
        render_mode (str, optional): Render mode. Defaults to "human".
        control_type (str, optional): "ee" to control end-effector position or "joints" to control joint values.
            Defaults to "ee".
        render_width (int, optional): Image width. Defaults to 720.
        render_height (int, optional): Image height. Defaults to 480.
    """

    def __init__(
        self,
        observation_type: int,
        objects_count: int = 5,
        blocker_bar: bool = True,
        render_mode: str = "human",
        control_type: str = "ee",
        renderer: str = "OpenGL",
        render_width: int = 720,
        render_height: int = 480,
        sorting_count: int = 1
    ) -> None:
        if observation_type not in [OBSERVATION_IMAGE, OBSERVATION_POSES]:
            raise ValueError("observation_type must be one of either images or poses")

        sim = PyBullet(
            render_mode=render_mode,
            background_color=np.array((200, 200, 200)),
            renderer=renderer,
        )
        robot = Panda(
            sim,
            block_gripper=False,
            base_position=np.array([-0.6, 0.0, 0.0]),
            control_type=control_type,
        )
        task = Pick_And_Place(sim,
                              observation_type,
                              robot,
                              objects_count=objects_count,
                              blocker_bar=blocker_bar,
                              sorting_count=sorting_count)
        super().__init__(
            robot,
            task,
            render_width=render_width,
            render_height=render_height,
            render_target_position=None,
            render_distance=0.9,
            render_yaw=45,
            render_pitch=-30,
            render_roll=0.0,
        )
        self.total_score = 0
        self.sim.place_visualizer(
            target_position=np.zeros(3), distance=0.9, yaw=45, pitch=-30
        )

    def reset(self) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray]]:
        with self.sim.no_rendering():
            self.robot.reset()
            self.task.reset()
        observation = self._get_obs()
        self.total_score = 0
        return observation, None

    def _get_obs(self) -> Dict[str, np.ndarray]:
        observation, _ = self.task.get_obs()
        observation = observation.astype(np.float32)
        achieved_goal = self.task.get_achieved_goal().astype(np.float32)
        return {
            "observation": observation,
            "achieved_goal": achieved_goal,
        }

    def get_obs(self) -> np.ndarray:
        observation, _ = self.task.get_obs()
        return observation.astype(np.float32)

    def step(
        self, action: np.ndarray
    ) -> Tuple[Dict[str, np.ndarray], float, bool, bool, Dict[str, Any]]:
        score_prior = self.task.score

        if isinstance(action, dict):
            discrete_action = action["discrete"]
            continuous_action = action["continuous"]
            self.robot.set_action(continuous_action)

        self.sim.step()
        observation = self._get_obs()
        score_after = self.task.score

        # An episode is terminated iff the agent has reached the target
        terminated = bool(
            self.task.is_success(observation["achieved_goal"], self.task.get_goal())
        )
        truncated = False
        info = {"is_success": terminated}
        step_penalty = STEP_PENALTY
        reward = (score_after - score_prior) + step_penalty
        self.total_score += reward
        # print("Score: ",self.total_score)
        # print("reward: ", reward)
        return observation, reward, terminated, truncated, info


SORTING_ONE = "sorting_one"
SORTING_TWO = "sorting_two"
SORTING_THREE = "sorting_three"
GOALS = [SORTING_ONE, SORTING_TWO, SORTING_THREE]

CUBE = 0
CYLINDER = 1
SPHERE = 2
SHAPES = [CUBE, CYLINDER, SPHERE]

# This is the expected correct sorting results
CORRECT_SORTS = {
    SORTING_ONE: CYLINDER,
    SORTING_TWO: SPHERE,
    SORTING_THREE: CUBE,
}


# FLOOR_PENALTY = -50
# # WRONG_SORT_REWARD = 25
# # SORT_REWARD = 100
# WRONG_SORT_REWARD = 200
# SORT_REWARD = 500

MOVE_TOWARD_OBJECT_REWARD = -1.0     # Reward for moving EE toward the object
GRASP_SUCCESS_REWARD = 50.0        # Reward for successful grasp
MOVE_OBJECT_TO_GOAL_REWARD = -1.0   # Reward for moving object toward goal
DROP_SUCCESS_REWARD = 50.0       # Reward for successfully placing in correct goal
WRONG_DROP_PENALTY = -20.0        # Penalty for placing object in wrong goal
FLOOR_COLLISION_PENALTY = -50.0   # Penalty for dropping the object on the floor
STEP_PENALTY = -0.1               # Small penalty to encourage efficiency
GRASP_THRESHOLD = 0.02

OBSERVATION_POSES: int = 0
OBSERVATION_IMAGE: int = 1


import time
def test_env():

    env = My_Arm_RobotEnv(observation_type=0,
                          render_mode="human",
                          blocker_bar=False,
                          objects_count=1,
                          sorting_count=2
                          )
    add_world_frame()
    observation, info = env.reset()

    for _ in range(10000):
        time.sleep(1/24)
        action = env.action_space.sample()
        print(action)
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            print("Run 1 episode")
            observation, info = env.reset()


import time


def test_fixed_actions():
    env = My_Arm_RobotEnv(
        observation_type=0,
        render_mode="human",
        blocker_bar=False,
        objects_count=1,
        sorting_count=2
    )

    observation, info = env.reset()

    # List of fixed actions to cycle through
    fixed_actions = [
        [0, 0, 0.1, 0],  # Move up
        [0, 0, -0.1, 0],  # Move down
        [-0.1, 0, 0, 0],  # Move left
        [0.1, 0, 0, 0],  # Move right
        [0, 0.1, 0, 0],  # Move forward
        [0, -0.1, 0, 0],  # Move backward
        [0, 0, 0, 0.3],  # Open gripper
        [0, 0, 0, -0.2],  # Close gripper
        [0, 0, 0.1, 0.7],  # Move up + open gripper
        [0, 0, -0.1, -0.5]  # Move down + close gripper
    ]
    frame = []
    for action in fixed_actions:
        for _ in range(50):  # Each action lasts for 50 time steps
            # frame.append(env.render('human'))
            observation, reward, terminated, truncated, info = env.step(action)
            print(f"Action: {action}, Reward: {reward}, Terminated: {terminated}")
            time.sleep(1 / 24)  # Delay for rendering

        if terminated or truncated:
            print("Episode ended, resetting environment.")
            observation, info = env.reset()

    # display.HTML(display_anim(frame))

# if __name__ == '__main__':
#     test_fixed_actions()


In [8]:
import torch
from torch import Tensor
from torch.nn import (Sequential, Module, Linear, ModuleList, Softplus,ModuleDict, ModuleList)
from torch.nn import LeakyReLU
import numpy as np
from typing import Union, List

class DiscreteActor(Module):
    def __init__(
            self,
            obs_dim: int = 20,
            output_dim: int = 3,
            control_type=None
    ):
        """Init the discrete actor. This network estimate a distribution of
        discrete actions.
        Args:
            obs_dim (int, optional): Dimension of observation space. Defaults to 20.
            output_size (int, optional): Output size or number of discrete
            actions. Defaults to 3 (Move, Pick, Place)
        """
        super(DiscreteActor, self).__init__()

        if control_type is not None and control_type == 'pendulum':
            obs_dim = 3
            output_dim = 1

        self.model = Sequential(
            Linear(obs_dim, 256),
            LeakyReLU(),
            Linear(256, 128),
            LeakyReLU(),
            Linear(128, 64),
            LeakyReLU(),
            Linear(64, output_dim),
        )

    def forward(self, input: Union[np.ndarray, Tensor, List]) -> Union[
        np.ndarray, Tensor, List]:
        if isinstance(input, np.ndarray):
            input_tensor: Tensor = torch.from_numpy(input.astype("float32"))
        elif type(input) is list:
            input_tensor: Tensor = torch.from_numpy(
                np.array(input).astype("float32"))
        else:
            input_tensor = input

        # return distribution
        output = self.model(input_tensor)
        output_dist = torch.distributions.Categorical(logits=output)
        return output_dist

    def save(self, filepath: str):
        torch.save({
            "model": self.model.state_dict(),
        }, filepath)

    def load(self, filepath: str):
        data = torch.load(filepath)
        self.model.load_state_dict(data["model"])


class ContinuousActor(Module):
    def __init__(
            self,
            obs_dim: int = 20,
            continuous_param_dim: List = [4, 4, 4],
            control_type=None
    ):
        """Init the continuous actor. This network predicts mean and std for
        the continuous parameters.
        Args:
            obs_dim (int, optional): Dimension of observation space. Defaults to 20.
            continuous_param_dim (int, optional): Dimension of continuous
            parameter. Defaults to [1, 1, 1, 1], meaning each discrete action only has 1 parameter
        """
        super(ContinuousActor, self).__init__()

        if control_type is not None and control_type == 'pendulum':
            obs_dim = 3
            continuous_param_dim = [1]

        self.model = ModuleList(
            ModuleDict({
                "mean": Sequential(
                    Linear(obs_dim, 64),
                    LeakyReLU(),
                    Linear(64, param_dim)
                ),
                "std": Sequential(
                    Linear(obs_dim, 64),
                    LeakyReLU(),
                    Linear(64, param_dim),
                    Softplus()  # Ensures positive standard deviations
                )
            })
            for param_dim in continuous_param_dim
        )

    def forward(self, input: Union[np.ndarray, Tensor, List]):
        if isinstance(input, np.ndarray):
            input_tensor: Tensor = torch.from_numpy(input.astype("float32"))
        elif type(input) is list:
            input_tensor: Tensor = torch.from_numpy(
                np.array(input).astype("float32"))
        else:
            input_tensor = input

        continuous_params = [
            {
                "mean": head["mean"](input_tensor),
                "std": head["std"](input_tensor)
            }
            for head in self.model
        ]

        return continuous_params


    def save(self, filepath: str):
        torch.save({
            "model": self.model.state_dict(),
        }, filepath)

    def load(self, filepath: str):
        data = torch.load(filepath)
        self.model.load_state_dict(data["model"])


class Critic(Module):
    def __init__(
            self,
            obs_dim: int,
            control_type=None
    ):
        """Init the critic network. This network estimate V(s)"""
        super(Critic, self).__init__()

        if control_type is not None and control_type == 'pendulum':
            obs_dim = 3

        self.model = Sequential(
            Linear(obs_dim, 128),
            LeakyReLU(),
            Linear(128, 64),
            LeakyReLU(),
            Linear(64, 32),
            LeakyReLU(),
            Linear(32, 1),
        )

    def forward(self, input: np.ndarray) -> Tensor:
        if isinstance(input, np.ndarray):
            input_tensor: Tensor = torch.from_numpy(input.astype("float32"))
        elif type(input) is list:
            input_tensor: Tensor = torch.from_numpy(
                np.array(input).astype("float32"))
        else:
            input_tensor = input

        return self.model(input_tensor)

    def save(self, filepath: str):
        torch.save({
            "model": self.model.state_dict(),
        }, filepath)

    def load(self, filepath: str):
        data = torch.load(filepath)
        self.model.load_state_dict(data["model"])



In [9]:
import torch
import numpy as np
from torch import Tensor
from torch.nn import MSELoss
from torch.distributions import Normal
from panda_gym.envs.core import RobotTaskEnv
from typing import List, Tuple
import sys
import matplotlib.pyplot as plt
import pickle


class Trainer:
    def __init__(
            self,
            env: RobotTaskEnv,
            discrete_actor: DiscreteActor,
            continuous_actor: ContinuousActor,
            critic: Critic,
            timesteps: int,
            timesteps_per_batch: int,
            max_timesteps_per_episode: int,
            training_cycles_per_batch: int = 5,
            gamma: float = 0.99,
            epsilon: float = 0.2,
            alpha: float = 3e-4,
            save_every_x_timesteps: int = 50000
    ):
        # Environment
        self.env = env

        # Neural networks
        self.discrete_actor = discrete_actor
        self.continuous_actor = continuous_actor
        self.critic = critic

        # Hypeparameters
        self.gamma = gamma
        self.epsilon = epsilon
        self.alpha = alpha

        # Iteration parameters
        self.timesteps = timesteps
        self.current_timestep = 0
        self.max_timesteps_per_episode = max_timesteps_per_episode
        self.timesteps_per_batch = timesteps_per_batch
        self.training_cycles_per_batch = training_cycles_per_batch
        self.save_every_x_timesteps = save_every_x_timesteps

        # Optimizers
        self.discrete_optimizer = torch.optim.Adam(
            params=self.discrete_actor.parameters(), lr=self.alpha)
        self.continuous_optimizer = torch.optim.Adam(
            params=self.continuous_actor.parameters(), lr=self.alpha)
        self.critic_optimizer = torch.optim.Adam(
            params=self.critic.parameters(),
                                                 lr=self.alpha)
        # Memory
        self.total_rewards: List[float] = []
        self.terminal_timesteps: List[int] = []
        self.discrete_actor_losses: List[float] = []
        self.continuous_actor_losses: List[float] = []
        self.critic_losses: List[float] = []
        self.previous_print_length: int = 0
        self.current_action = "Initializing"
        self.last_save: int = 0

    def print_status(self):
        latest_reward = 0.0
        average_reward = 0.0
        best_reward = 0.0

        latest_discrete_loss = 0.0
        avg_discrete_loss = 0.0
        latest_continuous_loss = 0.0
        avg_continuous_loss = 0.0

        latest_critic_loss = 0.0
        avg_critic_loss = 0.0
        recent_change = 0.0

        if len(self.total_rewards) > 0:
            latest_reward = self.total_rewards[-1]

            last_n_episodes = 100
            average_reward = np.mean(self.total_rewards[-last_n_episodes:])

            episodes = [
                i
                for i in range(
                    len(self.total_rewards[-last_n_episodes:]),
                    min(last_n_episodes, 0),
                    -1,
                )
            ]
            coefficients = np.polyfit(
                episodes,
                self.total_rewards[-last_n_episodes:],
                1,
            )
            recent_change = coefficients[0]

            best_reward = max(self.total_rewards)

        if len(self.discrete_actor_losses) > 0:
            avg_count = 3 * self.timesteps_per_batch
            latest_discrete_loss = self.discrete_actor_losses[-1]
            avg_discrete_loss = np.mean(
                self.discrete_actor_losses[-avg_count:])
            latest_continuous_loss = self.continuous_actor_losses[-1]
            avg_continuous_loss = np.mean(
                self.continuous_actor_losses[-avg_count:])
            latest_critic_loss = self.critic_losses[-1]
            avg_critic_loss = np.mean(self.critic_losses[-avg_count:])

        msg = f"""
            =========================================
            Timesteps: {self.current_timestep:,} / {self.timesteps:,} ({round((self.current_timestep / self.timesteps) * 100, 4)}%)
            Episodes: {len(self.total_rewards):,}
            Currently: {self.current_action}
            Latest Reward: {round(latest_reward)}
            Latest Avg Rewards: {round(average_reward)}
            Recent Change: {round(recent_change, 2)}
            Best Reward: {round(best_reward, 2)}
            Latest Discrete Actor Loss: {round(latest_discrete_loss, 4)}
            Latest Continuous Actor Loss: {round(latest_continuous_loss, 4)}
            Avg Discrete Actor Loss: {round(avg_discrete_loss, 4)}
            Avg Continuous Actor Loss: {round(avg_continuous_loss, 4)}
            Latest Critic Loss: {round(latest_critic_loss, 4)}
            Avg Critic Loss: {round(avg_critic_loss, 4)}
            =========================================
        """

        # We print to STDERR as a hack to get around the noisy pybullet
        # environment. Hacky, but effective if paired w/ 1> /dev/null
        print(msg, file=sys.stderr)

    def create_plot(self, filepath: str):
        last_n_episodes = 10

        episodes = [i + 1 for i in range(len(self.total_rewards))]
        averages = [
            np.mean(self.total_rewards[i - last_n_episodes: i])
            for i in range(len(self.total_rewards))
        ]
        trend_data = np.polyfit(episodes, self.total_rewards, 1)
        trendline = np.poly1d(trend_data)

        plt.scatter(
            episodes, self.total_rewards, color="green"
        )  # , linestyle='None', marker='o', color='green')
        plt.plot(episodes, averages, linestyle="solid", color="red")
        plt.plot(episodes, trendline(episodes), linestyle="--", color="blue")

        plt.title("Rewards per episode")
        plt.ylabel("Reward")
        plt.xlabel("Episode")
        plt.savefig(filepath)

    def save(self, directory: str):
        """
        save will save the models, state, and any additional
        data to the given directory
        """
        self.last_save = self.current_timestep

        self.discrete_actor.save(f"{directory}/discrete_actor.pth")
        self.continuous_actor.save(f"{directory}/continuous_actor.pth")
        self.critic.save(f"{directory}/critic.pth")
        self.create_plot(f"{directory}/rewards.png")

        # Now save the trainer's state data
        data = {
            "timesteps": self.timesteps,
            "current_timestep": self.current_timestep,
            "max_timesteps_per_episode": self.max_timesteps_per_episode,
            "timesteps_per_batch": self.timesteps_per_batch,
            "save_every_x_timesteps": self.save_every_x_timesteps,
            "γ": self.gamma,
            "ε": self.epsilon,
            "α": self.alpha,
            "training_cycles_per_batch": self.training_cycles_per_batch,
            "total_rewards": self.total_rewards,
            "terminal_timesteps": self.terminal_timesteps,
            "discrete_actor_losses": self.discrete_actor_losses,
            "continuous_actor_losses": self.continuous_actor_losses,
            "critic_losses": self.critic_losses,
        }
        pickle.dump(data, open(f"{directory}/state.data", "wb"))

    def load(self, directory: str):
        """
        Load will load the models, state, and any additional
        data from the given directory
        """
        # Load our models first; they're the simplest
        self.discrete_actor.load(f"{directory}/discrete_actor.pth")
        self.continuous_actor.load(f"{directory}/continuous_actor.pth")
        self.critic.load(f"{directory}/critic.pth")

        data = pickle.load(open(f"{directory}/state.data", "rb"))

        self.timesteps = data["timesteps"]
        self.current_timestep = data["current_timestep"]
        self.last_save = self.current_timestep
        self.max_timesteps_per_episode = data["max_timesteps_per_episode"]
        self.timesteps_per_batch = data["timesteps_per_batch"]
        self.save_every_x_timesteps = data["save_every_x_timesteps"]

        # Hyperparameters
        self.gamma = data["γ"]
        self.epsilon = data["ε"]
        self.alpha = data["α"]
        self.training_cycles_per_batch = data["training_cycles_per_batch"]

        # Memory
        self.total_rewards = data["total_rewards"]
        self.terminal_timesteps = data["terminal_timesteps"]
        self.discrete_actor_losses = data["discrete_actor_losses"]
        self.continuous_actor_losses = data["continuous_actor_losses"]
        self.critic_losses = data["critic_losses"]

        self.discrete_optimizer = torch.optim.Adam(
            self.discrete_actor.parameters(), lr=self.alpha)
        self.continuous_optimizer = torch.optim.Adam(
            self.continuous_actor.parameters(), lr=self.alpha)
        self.critic_optimizer = torch.optim.Adam(
            self.critic.parameters(), lr=self.alpha)

    def run_episode(self):
        """Run a single episode."""
        observation, _ = self.env.reset()
        if isinstance(observation, dict):
            observation = observation["observation"]

        timesteps = 0
        observations = []
        discrete_actions = []
        continuous_params = []
        discrete_log_probs = []
        continuous_log_probs = []
        rewards = []

        while True:
            timesteps += 1

            observations.append(observation)

            current_discrete_dist = self.discrete_actor(observation)
            current_discrete_action = current_discrete_dist.sample()
            current_discrete_log_prob = current_discrete_dist.log_prob(
                current_discrete_action).detach().numpy()
            current_discrete_action = current_discrete_action.detach().numpy()

            current_continuous_params = self.continuous_actor(observation)
            mean = current_continuous_params[current_discrete_action][
                'mean']
            std = current_continuous_params[current_discrete_action][
                'std']
            current_continuous_dist = torch.distributions.Normal(mean, std)
            current_continuous_action = current_continuous_dist.sample()
            continuous_log_prob = current_continuous_dist.log_prob(
                current_continuous_action).detach().numpy()
            current_continuous_action = current_continuous_action.detach().numpy()

            action = {
                'discrete': current_discrete_action,
                'continuous': current_continuous_action
            }

            obs, reward, terminated, _, _ = self.env.step(action)


            discrete_actions.append(current_discrete_action)
            discrete_log_probs.append(current_discrete_log_prob)

            continuous_params.append(current_continuous_action)
            continuous_log_probs.append(continuous_log_prob)

            rewards.append(reward)

            if timesteps >= self.max_timesteps_per_episode:
                terminated = True

            if terminated:
                break

        # Calculate the discounted rewards for this episode
        discounted_rewards = self.calculate_discounted_reward(rewards)

        # Get the terminal reward and record for status tracking
        self.total_rewards.append(sum(rewards))

        return (observations, discrete_actions, continuous_params,
                discrete_log_probs, continuous_log_probs, discounted_rewards)

    def rollout(self):
        """Perform a rollout of the environment and return the memory of the
        episode with the current actor models
        """
        observations = []
        discrete_log_probabilities = []
        continuous_log_probabilities = []
        discrete_actions = []
        continuous_actions = []
        rewards = []

        while len(observations) < self.timesteps_per_batch:
            self.current_action = "Rollout"
            (
                obs,
                chosen_discrete_actions,
                chosen_continuous_actions,
                discrete_log_probs,
                continuous_log_probs,
                rwds
            ) = self.run_episode()

            # Combine these arrays into overall batch
            observations += obs
            discrete_actions += chosen_discrete_actions
            continuous_actions += chosen_continuous_actions
            discrete_log_probabilities += discrete_log_probs
            continuous_log_probabilities += continuous_log_probs
            rewards += rwds

            # Increment count of timesteps
            self.current_timestep += len(obs)

            self.print_status()

        # Trim the batch memory to the batch size
        observations = observations[: self.timesteps_per_batch]
        discrete_actions = discrete_actions[: self.timesteps_per_batch]
        continuous_actions = continuous_actions[: self.timesteps_per_batch]
        discrete_log_probabilities = discrete_log_probabilities[
            : self.timesteps_per_batch]
        continuous_log_probabilities = continuous_log_probabilities[
            : self.timesteps_per_batch]
        rewards = rewards[: self.timesteps_per_batch]

        return (observations, discrete_actions, continuous_actions,
                discrete_log_probabilities, continuous_log_probabilities,
                rewards)

    def calculate_discounted_reward(self, rewards):
        """Calculate the discounted reward of each timestep of an episode
        given its initial rewards and episode length"""
        discounted_rewards = []
        discounted_reward = 0.0
        for reward in reversed(rewards):
            discounted_reward = reward + self.gamma * discounted_reward
            discounted_rewards.insert(0, discounted_reward)

        return discounted_rewards

    def calculate_normalized_advantage(self, observations, rewards):
        """Calculate the normalized advantage of each timestep of a given
        batch of episode """
        V = self.critic(observations).detach().squeeze()

        advantage = Tensor(np.array(rewards, dtype="float32")) - V
        normalized_advantage = (advantage - advantage.mean()) / (
            advantage.std() + 1e-8)

        return normalized_advantage

    def training_step(
            self,
            observations,
            discrete_actions,
            continuous_actions,
            discrete_log_probabilities,
            continuous_log_probabilities,
            rewards,
            normalized_advantage
    ):
        """Peform a single epoch of training for the actors and critic model. Return the loss for each model at the end of the step"""
        # ---- Discrete Actor ----
        current_discrete_dist = self.discrete_actor(observations)
        current_discrete_log_probs = current_discrete_dist.log_prob(
            discrete_actions)
        discrete_ratio = torch.exp(
            current_discrete_log_probs - discrete_log_probabilities)
        discrete_actor_loss = -torch.min(
            discrete_ratio * normalized_advantage,
            torch.clamp(discrete_ratio, 1 - self.epsilon, 1 +
                        self.epsilon) * normalized_advantage
        ).mean()

        self.discrete_optimizer.zero_grad()
        discrete_actor_loss.backward()
        self.discrete_optimizer.step()

        # ---- Continuous Actor ----
        current_continuous_params = self.continuous_actor(observations)
        means = [current_continuous_params[
                                 int(discrete_action.item())]['mean']
                             for discrete_action in discrete_actions]
        stds = [current_continuous_params[
                     int(discrete_action.item())]['std']
                 for discrete_action in discrete_actions]

        means = torch.stack(means)
        stds = torch.stack(stds)

        current_continuous_dist = torch.distributions.Normal(means, stds)
        current_continuous_log_probs = current_continuous_dist.log_prob(
            continuous_actions)
        continuous_ratios = torch.exp(
            current_continuous_log_probs - continuous_log_probabilities)

        normalized_advantage = normalized_advantage.unsqueeze(1).unsqueeze(2)
        continuous_actor_loss = -torch.min(
            continuous_ratios * normalized_advantage,
            torch.clamp(continuous_ratios, 1 - self.epsilon,
                        1 + self.epsilon) * normalized_advantage
        ).mean()

        self.continuous_optimizer.zero_grad()
        continuous_actor_loss.backward()
        self.continuous_optimizer.step()

        # ---- Critic Network ----
        V = self.critic(observations)
        reward_tensor = Tensor(rewards).unsqueeze(-1)
        critic_loss = MSELoss()(V, reward_tensor)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        return discrete_actor_loss.item(), continuous_actor_loss.item(), critic_loss.item()

    def train(self):
        while self.current_timestep <= self.timesteps:
            # Rollout to get next training batch
            observations, discrete_actions, continuous_actions, discrete_log_probabilities, continuous_log_probabilities, rewards = self.rollout()

            # Convert to numpy arrays and then to tensors
            observations = Tensor(np.array(observations, dtype=np.float32))
            discrete_actions = Tensor(
                np.array(discrete_actions, dtype=np.float32))
            continuous_actions = Tensor(
                np.array(continuous_actions, dtype=np.float32))
            discrete_log_probabilities = Tensor(
                np.array(discrete_log_probabilities, dtype=np.float32))
            continuous_log_probabilities = Tensor(
                np.array(continuous_log_probabilities, dtype=np.float32))
            rewards = Tensor(np.array(rewards, dtype=np.float32))

            # Perform training steps
            for c in range(self.training_cycles_per_batch):
                self.current_action = (
                    f"Training cycle {c+1}/{self.training_cycles_per_batch}"
                )
                self.print_status()
                # Calculate losses
                normalized_advantage = self.calculate_normalized_advantage(
                    observations, rewards)
                discrete_loss, continuous_loss, critic_loss = self.training_step(
                    observations, discrete_actions, continuous_actions, discrete_log_probabilities, continuous_log_probabilities, rewards, normalized_advantage)

                self.discrete_actor_losses.append(discrete_loss)
                self.continuous_actor_losses.append(continuous_loss)
                self.critic_losses.append(critic_loss)

            # Every x timesteps, save current status
            if self.current_timestep - self.last_save >= self.save_every_x_timesteps:
                self.current_action = "Saving"
                self.print_status()
                self.save("training")

        print("")
        print("Training complete!")
        self.save("training")

In [ ]:
# from test_env import *
# from model import DiscreteActor, ContinuousActor, Critic
import torch
# from trainer import Trainer

MOVE = 0
PICK = 1
PLACE = 2

action_space = {
    'discrete': {'Move': 0, 'Pick': 1, 'Place': 2},
    'continuous': [4, 4, 4]
}

discrete_dim = len(action_space['discrete'])
continuous_dim = action_space['continuous']

env = My_Arm_RobotEnv(
    observation_type=0,
    render_mode='rgb_array',
    blocker_bar=True,
    objects_count=2,
    sorting_count=3,
    renderer = "Tiny",
)
import os

# Create a directory named 'training' in the current working directory
os.makedirs('training', exist_ok=True)

# Verify if the directory was created
print("Directory 'training' created successfully" if os.path.exists('training') else "Failed to create directory")

obs, _ = env.reset()
obs_dim = len(obs['observation'])
d_actor = DiscreteActor(obs_dim=obs_dim, output_dim=discrete_dim)
c_actor = ContinuousActor(obs_dim=obs_dim,
                          continuous_param_dim=continuous_dim)
critic = Critic(obs_dim=obs_dim)

trainer = Trainer(
    env=env,
    discrete_actor=d_actor,
    continuous_actor=c_actor,
    critic=critic,
    timesteps=2_000_000,
    timesteps_per_batch=5_000,
    max_timesteps_per_episode=750,
)

trainer.train()

Directory 'training' created successfully
Object 0 incorrectly sorted into sorting_two


/tmp/ipykernel_23/1860987861.py:96: RankWarning: Polyfit may be poorly conditioned
  coefficients = np.polyfit(

            Timesteps: 750 / 2,000,000 (0.0375%)
            Episodes: 1
            Currently: Rollout
            Latest Reward: -272
            Latest Avg Rewards: -272
            Recent Change: -136.08
            Best Reward: -272.16
            Latest Discrete Actor Loss: 0.0
            Latest Continuous Actor Loss: 0.0
            Avg Discrete Actor Loss: 0.0
            Avg Continuous Actor Loss: 0.0
            Latest Critic Loss: 0.0
            Avg Critic Loss: 0.0
        


argv[0]=--background_color_red=0.7843137383460999
argv[1]=--background_color_green=0.7843137383460999
argv[2]=--background_color_blue=0.7843137383460999
b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed



            Timesteps: 1,500 / 2,000,000 (0.075%)
            Episodes: 2
            Currently: Rollout
            Latest Reward: -268
            Latest Avg Rewards: -270
            Recent Change: -4.32
            Best Reward: -267.84
            Latest Discrete Actor Loss: 0.0
            Latest Continuous Actor Loss: 0.0
            Avg Discrete Actor Loss: 0.0
            Avg Continuous Actor Loss: 0.0
            Latest Critic Loss: 0.0
            Avg Critic Loss: 0.0
        

            Timesteps: 2,250 / 2,000,000 (0.1125%)
            Episodes: 3
            Currently: Rollout
            Latest Reward: -432
            Latest Avg Rewards: -324
            Recent Change: 79.92
            Best Reward: -267.84
            Latest Discrete Actor Loss: 0.0
            Latest Continuous Actor Loss: 0.0
            Avg Discrete Actor Loss: 0.0
            Avg Continuous Actor Loss: 0.0
            Latest Critic Loss: 0.0
            Avg Critic Loss: 0.0
        

            

Object 0 dropped to the floor
Object 1 dropped to the floor
b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed
b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed


...


            Timesteps: 319,054 / 2,000,000 (15.9527%)
            Episodes: 480
            Currently: Rollout
            Latest Reward: -282
            Latest Avg Rewards: -288
            Recent Change: -0.44
            Best Reward: -108.08
            Latest Discrete Actor Loss: 0.0005
            Latest Continuous Actor Loss: -0.0
            Avg Discrete Actor Loss: -0.0001
            Avg Continuous Actor Loss: 0.0007
            Latest Critic Loss: 220.9577
            Avg Critic Loss: 886.7429
        


b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed
Object 0 dropped to the floor
Object 1 dropped to the floor
b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed
b3Warning[examples/SharedMemory/PhysicsDirect.cpp,866]:
Remove body failed



            Timesteps: 319,179 / 2,000,000 (15.9589%)
            Episodes: 481
            Currently: Rollout
            Latest Reward: -139
            Latest Avg Rewards: -287
            Recent Change: -0.56
            Best Reward: -108.08
            Latest Discrete Actor Loss: 0.0005
            Latest Continuous Actor Loss: -0.0
            Avg Discrete Actor Loss: -0.0001
            Avg Continuous Actor Loss: 0.0007
            Latest Critic Loss: 220.9577
            Avg Critic Loss: 886.7429
        

            Timesteps: 319,929 / 2,000,000 (15.9965%)
            Episodes: 482
            Currently: Rollout
            Latest Reward: -473
            Latest Avg Rewards: -289
            Recent Change: -0.47
            Best Reward: -108.08
            Latest Discrete Actor Loss: 0.0005
            Latest Continuous Actor Loss: -0.0
            Avg Discrete Actor Loss: -0.0001
            Avg Continuous Actor Loss: 0.0007
            Latest Critic Loss: 220.9577
       

Object 0 dropped to the floor



            Timesteps: 322,929 / 2,000,000 (16.1465%)
            Episodes: 486
            Currently: Rollout
            Latest Reward: -334
            Latest Avg Rewards: -291
            Recent Change: -0.37
            Best Reward: -108.08
            Latest Discrete Actor Loss: 0.0005
            Latest Continuous Actor Loss: -0.0
            Avg Discrete Actor Loss: -0.0001
            Avg Continuous Actor Loss: 0.0007
            Latest Critic Loss: 220.9577
            Avg Critic Loss: 886.7429
        

            Timesteps: 322,929 / 2,000,000 (16.1465%)
            Episodes: 486
            Currently: Training cycle 1/5
            Latest Reward: -334
            Latest Avg Rewards: -291
            Recent Change: -0.37
            Best Reward: -108.08
            Latest Discrete Actor Loss: 0.0005
            Latest Continuous Actor Loss: -0.0
            Avg Discrete Actor Loss: -0.0001
            Avg Continuous Actor Loss: 0.0007
            Latest Critic Loss: 220.9